# ANN with Keras and hyper-parameters exploration

In [1]:
import tensorflow as tf
from tensorflow import keras

tf.logging.set_verbosity(tf.logging.ERROR)

/home/alexander/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/alexander/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/alexander/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/alexander/.local/lib/python3.6/site-packages/tensorflow/python/framework/dty

We will use the library [deepreplay](https://github.com/dvgodoy/deepreplay) and some examples from its creator to visualise the ANN. However, they are curently some issues in the official version with the tensorflow v1.14 backend so please install from the following repository.

In [2]:
!pip install git+https://github.com/dackerer/deepreplay

  Cloning https://github.com/dackerer/deepreplay to /tmp/pip-gBalV4-build
  Using cached https://files.pythonhosted.org/packages/32/6b/0368cfa5e1d1ae169ab7dc78addda3fd5e6262e48d7373a9114bac7caff7/matplotlib-2.2.4-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d7/b1/3367ea1f372957f97a6752ec725b87886e12af1415216feec9067e31df70/numpy-1.16.5-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/12/90/3216b8f6d69905a320352a9ca6802a8e39fdb1cd93133c3d4163db8d5f19/h5py-2.10.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ad/fd/6bfe87920d7f4fd475acd28500a42482b6b84479832bdc0fe9e589a60ceb/Keras-2.3.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/31/9f/042db462417451e81035c3d43b722e88450c628a33dfda69777a801b0d40/scikit_learn-0.20.4-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e

Command "/usr/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-build-8B6tuI/subprocess32/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /tmp/pip-DH5K5O-record/install-record.txt --single-version-externally-managed --compile --user --prefix=" failed with error code 1 in /tmp/pip-build-8B6tuI/subprocess32/


Let's generate some labelled data, and some plot function for later.

In [3]:
from deepreplay.datasets.ball import load_data
from deepreplay.callbacks import ReplayData
from deepreplay.replay import Replay
from deepreplay.plot import compose_plots
import matplotlib.pyplot as plt

# number of features
n_input = 2
# generate data
X, Y = load_data(n_dims=n_input)

# Plot points
plt.scatter(X[:,0], X[:,1], c=Y.squeeze())
plt.show()

def plot_model(filename, group_name):
    '''
    plot function from: https://github.com/dvgodoy/deepreplay
    '''
    replay = Replay(replay_filename=filename, group_name=group_name)
    fig = plt.figure(figsize=(12, 6))
    ax_zvalues = plt.subplot2grid((2, 2), (0, 0))
    ax_weights = plt.subplot2grid((2, 2), (0, 1))
    ax_activations = plt.subplot2grid((2, 2), (1, 0))
    ax_gradients = plt.subplot2grid((2, 2), (1, 1))
    wv = replay.build_weights(ax_weights)
    gv = replay.build_gradients(ax_gradients)
    zv = replay.build_outputs(ax_zvalues, before_activation=True, 
                              exclude_outputs=True, include_inputs=False)
    av = replay.build_outputs(ax_activations, exclude_outputs=True, include_inputs=False)
    fig = compose_plots([zv, wv, av, gv], epoch=0, title=group_name)
    return fig

Using TensorFlow backend.


<Figure size 640x480 with 1 Axes>

We now create a function returning a Keras model, to be used multiple time.

In [4]:
from keras.models import Sequential
from keras.layers import Dense

def build_model(n_input, activation, initializer, optimizer='sgd', units=100):
    # Build a sequential ANN model  
    model = Sequential([
        Dense(units=units, activation=activation, input_shape=(n_input,), kernel_initializer=initializer, name='h1'),
        Dense(units=units, activation=activation, kernel_initializer=initializer, name='h2'),
        Dense(units=units, activation=activation, kernel_initializer=initializer, name='h3'),
        Dense(units=units, activation=activation, kernel_initializer=initializer, name='h4'),
        Dense(units=units, activation=activation, kernel_initializer=initializer, name='h5'),
        Dense(units=1, activation='sigmoid', kernel_initializer=initializer, name='o')
    ])
    '''
    # can also build a sequential ANN as follows
    model = Sequential()
    layer1 = Dense(...)
    model.add(layer1)
    # and so on, see https://keras.io/getting-started/sequential-model-guide/
    '''
    # Compiles the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['acc'])
    return model

Set a basic configuration, build, and display the model.

In [5]:
## Configuration
activation = 'sigmoid'
stddev_value = 0.01
initializer = keras.initializers.normal(mean=0, stddev=stddev_value)
group_name = 'activ_' + activation +'_init_normal_stdev_' + str(stddev_value)

# Model
model = build_model(n_input=n_input, activation=activation, initializer=initializer)

# Display the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
h1 (Dense)                   (None, 100)               300       
_________________________________________________________________
h2 (Dense)                   (None, 100)               10100     
_________________________________________________________________
h3 (Dense)                   (None, 100)               10100     
_________________________________________________________________
h4 (Dense)                   (None, 100)               10100     
_________________________________________________________________
h5 (Dense)                   (None, 100)               10100     
_________________________________________________________________
o (Dense)                    (None, 1)                 101       
Total params: 40,801
Trainable params: 40,801
Non-trainable params: 0
__________________________________________________

We now display some information about our ANN layers.

In [6]:
filename = 'weight_init.h5'
replaydata = ReplayData(X, Y, filename=filename, group_name=group_name, model=model)
fig = plot_model(filename, group_name)

ValueError: Unable to create group (name already exists)

## Exercises

Hyper-parameters exploration
* increase the initializer standard deviation to 0.1, then to 1.0
* same as above with different activation function (centered at zero in particular)
* try with glorot (normal and uniform)
* same as above with the ReLU activation function